In [7]:
pip install ultralytics


In [8]:
import cv2
import numpy as np
from ultralytics import YOLO

# --------- Load YOLOv8 Model ---------
model = YOLO("yolov8n.pt")  # or yolov8s.pt for better accuracy

# --------- Define Restricted Area ---------
# Coordinates: [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
restricted_area = np.array([(50, 300), (300, 300), (300, 450), (50, 450)])

# --------- Load Video ---------
video_path = r"D:\major_project_sus\preprocessed_dataset\Assault\Assault002_x264_3\10.pt"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]  # Inference

    # Draw restricted zone
    cv2.polylines(frame, [restricted_area], isClosed=True, color=(0, 0, 255), thickness=2)
    cv2.putText(frame, "Restricted Area", (restricted_area[0][0], restricted_area[0][1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        if cls_id == 0 and conf > 0.5:  # Person class
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  # Center of box

            # Check if center point is in restricted area
            inside = cv2.pointPolygonTest(restricted_area, (cx, cy), False) >= 0

            color = (0, 255, 0) if not inside else (0, 0, 255)
            label = "Person"
            if inside:
                label += " - 🚨 Intrusion"

            # Draw box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.circle(frame, (cx, cy), 4, color, -1)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("YOLOv8 - Intrusion Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [9]:
import cv2
import numpy as np
from ultralytics import YOLO

# --------- Load YOLOv8 Model ---------
model = YOLO("yolov8n.pt")  # You can change this to yolov8s.pt or any other

# --------- Define Restricted Area Polygon (customize based on your video) ---------
restricted_area = np.array([(50, 300), (300, 300), (300, 450), (50, 450)])  # Example area

# --------- Load Video ---------
video_path = r"D:\major_project_sus\preprocessed_dataset\Assault\Assault002_x264_3\10.pt"  # Replace with your test video
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    # Draw restricted zone
    cv2.polylines(frame, [restricted_area], isClosed=True, color=(0, 0, 255), thickness=2)
    cv2.putText(frame, "Restricted Area", (restricted_area[0][0], restricted_area[0][1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        if cls_id == 0 and conf > 0.5:  # Person class
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            inside = cv2.pointPolygonTest(restricted_area, (cx, cy), False) >= 0
            color = (0, 255, 0) if not inside else (0, 0, 255)
            label = "Person" + (" - 🚨 Intrusion" if inside else "")

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.circle(frame, (cx, cy), 4, color, -1)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("YOLOv8 Intrusion Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [10]:
from ultralytics import YOLO
import cv2
import time

# -------- Configuration --------
video_path = r"D:\major_project_sus\DCSASS Dataset\Assault\Assault049_x264.mp4\Assault049_x264_21.mp4"
model_path = "yolov8l.pt"  # Large model, change to yolov8n.pt or yolov8s.pt for faster performance

# -------- Load YOLOv8 Model --------
model = YOLO(model_path)

# -------- OpenCV Video Capture --------
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("❌ Error opening video file.")
    exit()

# -------- Frame-by-frame Processing --------
alert_triggered = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO Inference on current frame
    results = model.predict(source=frame, conf=0.4, classes=None, stream=True)

    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = model.names[cls_id]

            # Draw bounding box
            xyxy = box.xyxy[0].cpu().numpy().astype(int)
            x1, y1, x2, y2 = xyxy
            label = f"{class_name} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Trigger alert if person or any suspicious class detected
            if class_name.lower() == "person":
                alert_triggered = True

    # Show the frame
    cv2.imshow("YOLOv8 Detection", frame)

    # Alert message
    if alert_triggered:
        print("🚨 ALERT: Suspicious activity or person detected!")
        alert_triggered = False  # Reset after showing

    # Press 'q' to exit early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# -------- Cleanup --------
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 132.5ms
Speed: 2.4ms preprocess, 132.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
🚨 ALERT: Suspicious activity or person detected!

0: 480x640 2 persons, 14.0ms
Speed: 2.2ms preprocess, 14.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
🚨 ALERT: Suspicious activity or person detected!

0: 480x640 2 persons, 13.9ms
Speed: 1.5ms preprocess, 13.9ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
🚨 ALERT: Suspicious activity or person detected!

0: 480x640 1 person, 14.0ms
Speed: 1.6ms preprocess, 14.0ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
🚨 ALERT: Suspicious activity or person detected!

0: 480x640 2 persons, 19.0ms
Speed: 1.6ms preprocess, 19.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
🚨 ALERT: Suspicious activity or person detected!

0: 480x640 2 persons, 13.6ms
Speed: 1.5ms preprocess, 13.6ms inference, 1.9ms postprocess per image at shape (1, 3, 480

In [11]:
import torch
from ultralytics import YOLO
from IPython.display import Video, display
from pathlib import Path
import os
from model_r2plus1d import R2Plus1D  # Your custom model definition

# -------- Configuration --------
clip_tensor_path = r"D:\major_project_sus\preprocessed_dataset\Assault\Assault002_x264_3\10.pt"
video_path = r"DCSASS Dataset/Assault/Assault022_x264.mp4/Assault022_x264_3.mp4"
yolo_model_path = "yolov8n.pt"  # Change if using a different YOLOv8 model
activity_model_path = r"D:\major_project_sus\best_r2plus1d_model.pth"

# -------- Device Setup --------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------- Activity Class Names --------
activity_class_names = [
    "Arrest", "Arson", "Assault", "Burglary", "Explosion",
    "Fighting", "Normal", "RoadAccidents", "Robbery", "Shoplifting",
    "Stealing", "Vandalism", "Abuse", "Trespassing"
]

# -------- Load R(2+1)D model --------
activity_model = R2Plus1D(num_classes=len(activity_class_names)).to(device)
activity_model.load_state_dict(torch.load(activity_model_path, map_location=device))
activity_model.eval()

# -------- Load YOLOv8 model --------
yolo_model = YOLO(yolo_model_path)

# -------- Run YOLOv8 on Original Video --------
yolo_results = yolo_model.predict(source=video_path, show=False, save=True, conf=0.4)

# Get saved video path
save_dir = Path(yolo_results[0].save_dir)
output_path = save_dir / Path(video_path).with_suffix('.avi').name
output_path = str(output_path)

# -------- Run R(2+1)D on Preprocessed Clip --------
clip_tensor = torch.load(clip_tensor_path)  # [T, C, H, W]
if clip_tensor.dim() == 4:
    clip_tensor = clip_tensor.unsqueeze(0).to(device)  # [1, T, C, H, W]

with torch.no_grad():
    output = activity_model(clip_tensor)
    activity_index = output.argmax(dim=1).item()
    predicted_activity = activity_class_names[activity_index]

# -------- Display Results --------
print("✅ YOLOv8 Detection Done.")
print(f"📁 Detection video saved at: {output_path}")
print(f"🎬 Predicted Activity: {predicted_activity}")

# 🚨 Raise Alert if Activity is Suspicious
if predicted_activity.lower() not in ["normal", "walking", "standing"]:
    print(f"🚨 ALERT: Suspicious activity detected → {predicted_activity}")

# -------- Display Output Video in Notebook --------
if os.path.exists(output_path):
    display(Video(filename=output_path, embed=True, width=720))
else:
    print("⚠️ Output video not found.")




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) d:\major_project_sus\DCSASS Dataset\Assault\Assault022_x264.mp4\Assault022_x264_3.mp4: 480x640 (no detections), 105.3ms
video 1/1 (frame 2/60) d:\major_project_sus\DCSASS Dataset\Assault\Assault022_x264.mp4\Assault022_x264_3.mp4: 480x640 (no detections), 10.0ms
video 1/1 (frame 3/60) d:\major_project_sus\DCSASS Dataset\Assault\Assault022_x264.mp4\Assault022_x264_3.mp4: 480x640 (no detections), 8.9ms
video 1/1 (frame 4/60) d:\major_

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 3 is not equal to len(dims) = 5

In [ ]:
import torch
from ultralytics import YOLO
from IPython.display import Video, display
from pathlib import Path
import os
from model_r2plus1d import R2Plus1D  # Your custom model definition

# -------- Configuration --------
clip_tensor_path = r"D:\major_project_sus\preprocessed_dataset\Assault\Assault002_x264_3\10.pt"
video_path = r"DCSASS Dataset/Assault/Assault022_x264.mp4/Assault022_x264_3.mp4"
yolo_model_path = "yolov8n.pt"  # Change if using a different YOLOv8 model
activity_model_path = r"D:\major_project_sus\best_r2plus1d_model.pth"

# -------- Device Setup --------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------- Activity Class Names --------
activity_class_names = [
    "Arrest", "Arson", "Assault", "Burglary", "Explosion",
    "Fighting", "Normal", "RoadAccidents", "Robbery", "Shoplifting",
    "Stealing", "Vandalism", "Abuse", "Trespassing"
]

# -------- Load R(2+1)D model --------
activity_model = R2Plus1D(num_classes=len(activity_class_names)).to(device)
activity_model.load_state_dict(torch.load(activity_model_path, map_location=device))
activity_model.eval()

# -------- Load YOLOv8 model --------
yolo_model = YOLO(yolo_model_path)

# -------- Run YOLOv8 on Original Video --------
yolo_results = yolo_model.predict(source=video_path, show=False, save=True, conf=0.4)

# Get saved video path
save_dir = Path(yolo_results[0].save_dir)
output_path = save_dir / Path(video_path).with_suffix('.avi').name
output_path = str(output_path)

# -------- Run R(2+1)D on Preprocessed Clip --------
clip_tensor = torch.load(clip_tensor_path)
print(f"Loaded clip tensor shape: {clip_tensor.shape}")

# Handle tensor dimensions properly
if clip_tensor.dim() == 4:
    clip_tensor = clip_tensor.unsqueeze(0)  # [1, T, C, H, W]
elif clip_tensor.dim() == 5:
    pass  # Already [1, T, C, H, W]
else:
    raise ValueError(f"Unexpected clip tensor shape: {clip_tensor.shape}")

clip_tensor = clip_tensor.to(device)

# -------- Make Prediction --------
with torch.no_grad():
    output = activity_model(clip_tensor)
    activity_index = output.argmax(dim=1).item()
    predicted_activity = activity_class_names[activity_index]

# -------- Display Results --------
print("✅ YOLOv8 Detection Done.")
print(f"📁 Detection video saved at: {output_path}")
print(f"🎬 Predicted Activity: {predicted_activity}")

# 🚨 Raise Alert if Activity is Suspicious
if predicted_activity.lower() not in ["normal", "walking", "standing"]:
    print(f"🚨 ALERT: Suspicious activity detected → {predicted_activity}")

# -------- Display Output Video in Notebook --------
if os.path.exists(output_path):
    display(Video(filename=output_path, embed=True, width=720))
else:
    print("⚠️ Output video not found.")




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) d:\major_project_sus\DCSASS Dataset\Assault\Assault022_x264.mp4\Assault022_x264_3.mp4: 480x640 (no detections), 14.6ms
video 1/1 (frame 2/60) d:\major_project_sus\DCSASS Dataset\Assault\Assault022_x264.mp4\Assault022_x264_3.mp4: 480x640 (no detections), 16.8ms
video 1/1 (frame 3/60) d:\major_project_sus\DCSASS Dataset\Assault\Assault022_x264.mp4\Assault022_x264_3.mp4: 480x640 (no detections), 27.0ms
video 1/1 (frame 4/60) d:\major_

ValueError: Unexpected clip tensor shape: torch.Size([3, 224, 224])

# start here for the new yolo


In [ ]:
import os
import shutil
import pandas as pd
import glob
import random

# Set random seed for consistent train/val split
random.seed(42)

# Paths
labels_dir = r"D:\major_project_sus\DCSASS Dataset\Labels"
frames_root = r"D:\major_project_sus\processed_dataset"
output_all = r"D:\major_project_sus\frame_classification\all"

# Create output folders
for label_folder in ['normal', 'suspicious']:
    os.makedirs(os.path.join(output_all, label_folder), exist_ok=True)

# Process all CSV files (e.g., Abuse.csv, Assault.csv, etc.)
csv_files = glob.glob(os.path.join(labels_dir, "*.csv"))

for csv_file in csv_files:
    df = pd.read_csv(csv_file, header=None, names=["clip_name", "activity", "label"])
    activity_name = os.path.basename(csv_file).replace(".csv", "")
    
    for idx, row in df.iterrows():
        clip_folder = row["clip_name"]
        label = row["label"]
        
        # Frame directory: D:/.../processed_dataset/Abuse/Abuse001_x264_0/
        frame_dir = os.path.join(frames_root, activity_name, clip_folder)
        if not os.path.exists(frame_dir):
            continue
        
        for frame_file in os.listdir(frame_dir):
            if frame_file.endswith(".jpg"):
                src = os.path.join(frame_dir, frame_file)
                dest_label = "suspicious" if label == 1 else "normal"
                dest_path = os.path.join(output_all, dest_label, f"{clip_folder}_{frame_file}")
                shutil.copy(src, dest_path)

print("✅ All frames sorted into 'normal' and 'suspicious'. Now splitting into train/val...")

# Now split 50% into train/val
output_train = r"D:\major_project_sus\frame_classification\train"
output_val = r"D:\major_project_sus\frame_classification\val"

for split_folder in [output_train, output_val]:
    for label in ['normal', 'suspicious']:
        os.makedirs(os.path.join(split_folder, label), exist_ok=True)

# Split frames
for label in ['normal', 'suspicious']:
    all_images = glob.glob(os.path.join(output_all, label, "*.jpg"))
    random.shuffle(all_images)
    
    split_idx = len(all_images) // 2
    train_images = all_images[:split_idx]
    val_images = all_images[split_idx:]
    
    for img in train_images:
        shutil.copy(img, os.path.join(output_train, label, os.path.basename(img)))
    
    for img in val_images:
        shutil.copy(img, os.path.join(output_val, label, os.path.basename(img)))

print("✅ 50% split into 'train' and 'val' folders complete.")


✅ All frames sorted into 'normal' and 'suspicious'. Now splitting into train/val...
✅ 50% split into 'train' and 'val' folders complete.


In [ ]:
import os
import glob

# Base paths
image_base = r"D:\major_project_sus\frame_classification"
label_base = r"D:\major_project_sus\frame_classification_labels"

splits = ["train", "val"]
classes = {"normal": 0, "suspicious": 1}

for split in splits:
    for cls_name, cls_id in classes.items():
        img_folder = os.path.join(image_base, split, cls_name)
        label_folder = os.path.join(label_base, split, cls_name)
        
        # Ensure the label directory exists
        os.makedirs(label_folder, exist_ok=True)
        
        images = glob.glob(os.path.join(img_folder, "*.jpg"))
        for img_path in images:
            img_name = os.path.basename(img_path)
            label_name = img_name.replace(".jpg", ".txt")
            label_path = os.path.join(label_folder, label_name)
            
            with open(label_path, "w") as f:
                # Write full-image bounding box
                f.write(f"{cls_id} 0.5 0.5 1.0 1.0\n")

print("✅ YOLO annotation files created for all images.")


✅ YOLO annotation files created for all images.


In [ ]:
import os
import shutil
from glob import glob

# Source label root
label_root = r"D:\major_project_sus\frame_classification_labels"
# Destination image root (YOLO expects .txt to be here)
image_root = r"D:\major_project_sus\frame_classification"

splits = ['train', 'val']
classes = ['normal', 'suspicious']

for split in splits:
    for cls in classes:
        label_dir = os.path.join(label_root, split, cls)
        image_dir = os.path.join(image_root, split, cls)

        txt_files = glob(os.path.join(label_dir, "*.txt"))
        for txt_file in txt_files:
            dest_file = os.path.join(image_dir, os.path.basename(txt_file))
            shutil.copy(txt_file, dest_file)

print("✅ Annotation .txt files copied next to image files.")


✅ Annotation .txt files copied next to image files.


YOLO training start

In [ ]:
!pip install ultralytics


In [ ]:
from ultralytics import YOLO

# Optional: Check version
import ultralytics
ultralytics.checks()


Ultralytics 8.3.103  Python-3.11.5 torch-2.1.0 CUDA:0 (NVIDIA RTX A5000, 24563MiB)
Setup complete  (36 CPUs, 127.6 GB RAM, 302.7/500.0 GB disk)


In [ ]:
model = YOLO("yolov8n.pt")  # Load a pre-trained YOLOv8 Nano model

model.train(
    data="D:/major_project_sus/data.yaml",  # Path to your data.yaml
    epochs=50,
    imgsz=640,
    batch=16  # Adjust based on your system
)


New https://pypi.org/project/ultralytics/8.3.109 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.103  Python-3.11.5 torch-2.1.0 CUDA:0 (NVIDIA RTX A5000, 24563MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:/major_project_sus/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_f

train: Scanning D:\major_project_sus\frame_classification\train\normal... 101931 images, 0 backgrounds, 0 corrupt: 100%|██████████| 101931/101931 [00:46<00:00, 2182.20it/s]


train: New cache created: D:\major_project_sus\frame_classification\train\normal.cache


val: Scanning D:\major_project_sus\frame_classification\val\normal... 101931 images, 0 backgrounds, 0 corrupt: 100%|██████████| 101931/101931 [02:22<00:00, 714.17it/s]


val: New cache created: D:\major_project_sus\frame_classification\val\normal.cache
Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      2.49G     0.1204      0.729     0.9266         11        640: 100%|██████████| 6371/6371 [09:43<00:00, 10.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3186/3186 [04:22<00:00, 12.14it/s]


                   all     101931     101931          0          0          0          0

50 epochs completed in 13.968 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.103  Python-3.11.5 torch-2.1.0 CUDA:0 (NVIDIA RTX A5000, 24563MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3186/3186 [08:31<00:00,  6.23it/s]


                   all     101931     101931      0.912      0.926      0.973      0.973
                normal      57315      57315      0.904      0.954      0.976      0.976
            suspicious      44616      44616       0.92      0.898       0.97       0.97
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000026122AF65D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [2]:
from ultralytics import YOLO

# Load your trained model
model = YOLO(r"D:\major_project_sus\runs\detect\train2\weights\best.pt")

# Path to your test video
video_path = r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4" # 🔁 Replace with actual video path
# video_path=r"D:\major_project_sus\DCSASS Dataset\Abuse\Abuse008_x264.mp4\Abuse008_x264_18.mp4"
# video_path =r"D:\major_project_sus\DCSASS Dataset\Arrest\Arrest003_x264.mp4\Arrest003_x264_16.mp4"
# video_path = r"D:\major_project_sus\DCSASS Dataset\Assault\Assault004_x264.mp4\Assault004_x264_19.mp4"
# video_path = r"D:\major_project_sus\DCSASS Dataset\Explosion\Explosion009_x264.mp4\Explosion009_x264_8.mp4"
# video_path = r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-18 at 11.32.45 PM.mp4"
# video_path = r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-19 at 12.47.24 AM.mp4"
# video_path = r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-19 at 12.54.02 AM.mp4"
# video_path = r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-19 at 1.18.06 AM.mp4"
# video_path = r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-19 at 1.18.25 AM.mp4"
# video_path= r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-19 at 12.47.24 AM.mp4"
# video_path= r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-26 at 3.17.58 PM.mp4"



# Run prediction
results = model.predict(
    source=video_path,
    imgsz=640,
    conf=0.001,
    data=r"D:\major_project_sus\data.yaml",
    save=True,
    save_txt=True
)

print("✅ Prediction complete! Check 'runs/detect/predict' for the output video.")




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/98) C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4: 640x384 2 normals, 1 suspicious, 115.4ms
video 1/1 (frame 2/98) C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4: 640x384 2 normals, 1 suspicious, 8.9ms
video 1/1 (frame 3/98) C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4: 640x384 2 normals, 1 suspicious, 9.2ms
video 1/1 (frame 4/98) C:\Users\admin\Downloads\WhatsApp Vid

In [1]:
from ultralytics import YOLO

# Load your trained model
model = YOLO(r"D:\major_project_sus\runs\detect\train2\weights\best.pt")

# Path to test video
video_path = r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-18 at 11.23.37 PM.mp4"

# Run prediction
results = model.predict(
    source=video_path,
    imgsz=640,
    conf=0.6,          # reasonable threshold to reduce false positives
    iou=0.4,           # moderate NMS IoU
    agnostic_nms=True, # suppress overlapping boxes regardless of class
    save=True,
    save_txt=True
)

print("✅ Prediction complete! Check 'runs/detect/predict' for the output video.")


KeyboardInterrupt: 

In [27]:
from ultralytics import YOLO
import cv2
import os

# Load the trained YOLOv8 model
model = YOLO("runs/detect/train2/weights/best.pt")

# Video path (change \ to / for OpenCV)
# video_path = r"D:\major_project_sus\DCSASS Dataset\Explosion\Explosion024_x264.mp4\Explosion024_x264_28.mp4"
video_path=r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4"

# Open the video
cap = cv2.VideoCapture(video_path)

# Define class names (based on your training data)
class_names = ['normal', 'suspicious']

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Inference on current frame
    results = model(frame, imgsz=640, conf=0.5)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls_id = int(box.cls[0])
            cls_name = class_names[cls_id]

            # Coordinates for bounding box
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            color = (0, 255, 0) if cls_name == 'normal' else (0, 0, 255)

            # Draw the box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, cls_name, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # 🚨 Print alert if 'suspicious' detected
            if cls_name == 'suspicious':
                print("🚨 ALERT: Suspicious activity detected in this frame!")

    # Display the frame
    cv2.imshow("Suspicious Activity Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 640x384 1 normal, 11.3ms
Speed: 3.5ms preprocess, 11.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 normal, 22.0ms
Speed: 2.8ms preprocess, 22.0ms inference, 7.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 normals, 14.7ms
Speed: 3.4ms preprocess, 14.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 normals, 9.4ms
Speed: 2.5ms preprocess, 9.4ms inference, 6.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 normals, 9.2ms
Speed: 2.7ms preprocess, 9.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 normals, 9.0ms
Speed: 2.8ms preprocess, 9.0ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 8.5ms
Speed: 3.2ms preprocess, 8.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 10.2ms
Speed: 2.4ms preprocess, 10.2ms inference, 2.2ms postprocess per image at shape

In [26]:
# Use raw string literal for the path
model = YOLO(r"D:\major_project_sus\runs\detect\train2\weights\best.pt")  # Using raw string

# Or use double backslashes
model = YOLO("D:\\major_project_sus\\runs\\detect\\train2\\weights\\best.pt")  # Using double backslashes

# Now run inference on your video
results = model.predict(
    source=r"C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4", 
    save=True, 
    conf=0.25
)




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/98) C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4: 640x384 2 normals, 126.0ms
video 1/1 (frame 2/98) C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4: 640x384 2 normals, 9.1ms
video 1/1 (frame 3/98) C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4: 640x384 2 normals, 16.5ms
video 1/1 (frame 4/98) C:\Users\admin\Downloads\WhatsApp Video 2025-04-17 at 3.34.13 PM.mp4: 640x384 